# BERT

## Dataset creation

For the dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("antonoof/avito-data")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'avito-data' dataset.
Path to dataset files: /kaggle/input/avito-data


In [ ]:
reviews = pd.read_csv("reviews.csv", sep="\t", encoding="utf-8")

In [ ]:
reviews= reviews[['review']]

In [ ]:
def trim_review(text, min_words=5, max_words=20):
    words = text.split()
    return " ".join(words[:max_words])

reviews['review'] = reviews['review'].apply(trim_review)

In [ ]:
reviews['review_no_spaces'] = reviews['review'].str.replace(" ", "", regex=False)

In [ ]:
reviews = reviews.iloc[:15000]

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
dataset = Dataset.from_pandas(reviews)
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
model_name = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(["fix spaces:" + text for text in examples["review_no_spaces"]], max_length=256, truncation=True)
    labels = tokenizer(examples["review"], max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-spaces",
    num_train_epochs=8,
    predict_with_generate=True,
    logging_dir="./logs",
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/tmp/ipython-input-2429944011.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Step,Training Loss
500,7.452300
1000,2.237700
1500,1.657200
2000,1.368300
2500,1.211300
3000,1.077000
3500,0.985900
4000,0.928600
4500,0.878200
5000,0.840300


Step,Training Loss
500,7.452300
1000,2.237700
1500,1.657200
2000,1.368300
2500,1.211300
3000,1.077000
3500,0.985900
4000,0.928600
4500,0.878200
5000,0.840300


KeyboardInterrupt: 

In [ ]:
text = "новыйдивандоставканедорого"
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_length=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

]: новый тёнка доставка недорого


In [ ]:
texts

Column(['совершеннонесоответствуютразмернойсетке,оказалисьмалы,продавецотказалсявернутьденьги...', 'Вообщеужас,тканьсоответствует,нуаформа....прямойквадратпросто', 'Ужаснаяюбкапришла!!!Онатонкая,ужасномятаяивоняет!!!Сверхунепрошита,разошлисьнитки...простокошмар!!!Какможноотправлятьлюдом', 'товарбылвскрыт.грязная.рукавакороткиеиоченьузкие.навыбросить.', 'Заказаланосочки8мая,сегодня16сентября.Доставкитакинебыло,носкинедошли.Жаль.'])

In [ ]:

from torch import cuda
device = "cuda" if cuda.is_available() else "cpu"

texts_test = texts["bigrams"]  # берем тестовые строки без пробелов

# генерация исправленных текстов
corrected_texts = []
for text in tqdm(texts_test):
    text = "fix spaces:" + text
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, max_length=64)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    corrected_texts.append(corrected)


  1%|▏         | 14/1006 [00:03<04:12,  3.93it/s]


KeyboardInterrupt: 

In [ ]:
corrected_texts

['цифр',
 'курила айбон 14 про',
 'ищу дом в Москве',
 'сдаю квартиру с талигой и техники',
 'новый диван доставка недорого',
 'отдам даром кошку',
 'работа в москве удален',
 'куплю телевизор Philips',
 'я ищу груз и в для переезда',
 'ремонт квартир под ключ',
 'коплю о Laptop',
 'ищу квартиру у',
 'новая микроволновка а',
 'трудно продам велосипед']

In [ ]:
texts

,text_no_spaces,bigrams
0,text_no_spaces,text_no_spaces
1,куплюайфон14про,куплю айфон 14 про
2,ищудомвПодмосковье,ищу дом в Подмосковье
3,сдаюквартирусмебельюитехникой,сдаю квартиру с мебелью и техникой
4,новыйдивандоставканедорого,новый диван доставка недорого
...,...,...
1001,Янеусну.,Я не усну.
1002,Весна-яуженегреюпио.,Весна- я уже не грею пи о.
1003,Весна-скоровырастеттрава.,Вес на-скоро вырастет трава.
1004,Весна-выпосмотрите,Весна- вы посмотрите


In [ ]:
д